In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("ventas_cafeterias_espana_bares.csv")

In [ ]:
df.shape


(301514, 9)

In [7]:
df.head(10)

,ticket_id,fecha_hora,ciudad,bar,producto,categoria,precio_unitario,cantidad,metodo_pago
0,1,2025-03-03 22:25:16,Móstoles,Lounge Móstoles Sur,Croissant,reposteria,NaN,3,Bizum
1,2,2025-02-27 20:13:33,Eivissa,Cervecería Eivissa Puerto,Refresco,bebida,3.97,2,Efectivo
2,3,2025-09-14 23:53:31,Alicante,Café Alicante Este,Sandwich mixto,comida,3.31,1,Tarjeta
3,4,2025-07-12 10:06:51,Eivissa,Bar Eivissa Plaza Mayor,Latte,bebida,1.68,1,Efectivo
4,5,2025-03-02 03:42:48,Getafe,Cervecería Getafe Río,Napolitana,reposteria,1.7,2,Tarjeta
5,6,2025-11-01 20:42:20,Alicante,Lounge Alicante Oeste,Bizcocho,reposteria,1.4,1,Tarjeta
6,7,2025-05-17 22:19:12,Burgos,Gastrobar Burgos Norte,Café solo,bebida,2.52,1,Efectivo
7,8,2025-05-16 14:19:55,Ourense,Bar Ourense Casco Viejo,Sandwich mixto,comida,3.05,1,Bizum
8,9,2025-05-03 17:49:58,Ponferrada,Bar Ponferrada Estación,Cortado,bebida,1.47,2,Bizum
9,10,2025-05-01 00:39:41,Madrid,Taberna La Dolores,Tortilla de patata,comida,5.77,1,Efectivo


# 📊 Análisis Exploratorio de Ventas en Cafeterías de España



## **1. Carga y comprensión del conjunto de datos**

En este apartado se realiza una **primera toma de contacto con los datos** de ventas de cafeterías en España.

- Carga del archivo `.csv` que contiene la información del estudio.
- Visualización de las primeras filas para comprender la estructura del dataset.
- Revisión del número de registros y variables disponibles.
- Identificación del tipo de datos de cada columna.
- Descripción general de las variables más relevantes para el análisis.

*Este paso es fundamental para entender qué información tenemos y cómo podemos trabajar con ella.*

## **2. Limpieza y preprocesamiento de los datos**

Antes de realizar cualquier análisis, es necesario **asegurar la calidad de los datos**.

- Detección de valores nulos o inconsistentes.
- Tratamiento de valores faltantes mediante eliminación o sustitución.
- Corrección de tipos de datos incorrectos (numéricos, fechas o categorías).
- Eliminación de posibles registros duplicados.
- Preparación del dataset para el análisis exploratorio y la visualización.

*Unos datos limpios garantizan conclusiones más fiables.*

## **3. Análisis univariante de las ventas**

Se analiza la variable de ventas de forma individual para comprender su comportamiento general.

- Estudio de la distribución de las ventas.
- Identificación de la frecuencia de los distintos rangos de valores.
- Detección visual de posibles valores atípicos.

**Gráficos utilizados:**
- Histogramas  
- Boxplots 

## **4. Comparación de distribuciones mediante multihistogramas**

En este apartado se comparan las ventas según una característica concreta del dataset.

- Análisis de cómo varía la distribución de las ventas entre distintos grupos.
- Comparación visual entre categorías relevantes.
- Identificación de diferencias significativas entre los grupos analizados.

**Gráficos utilizados:**
- Multihistogramas (Matplotlib)

## **5. Evolución temporal de las ventas**

Se estudia cómo evolucionan las ventas a lo largo del tiempo.

- Análisis de la tendencia general de las ventas.
- Detección de picos, caídas o comportamientos repetitivos.
- Observación de posibles patrones estacionales.

**Gráficos utilizados:**
- Gráfico de líneas  

## **6. Relación entre variables numéricas**

Se analiza la relación entre las ventas y otras variables numéricas del dataset.

- Estudio de posibles relaciones o dependencias entre variables.
- Identificación de patrones de comportamiento conjunto.
- Comparación de distintas relaciones dentro de una misma visualización.

**Gráficos utilizados:**
- Scatter (gráfico de dispersión)  
- Scatter múltiple  

## **7. Análisis de correlación entre variables**

En este apartado se evalúa el grado de relación entre las variables numéricas.

- Cálculo de la correlación entre variables.
- Identificación de relaciones positivas, negativas o inexistentes.
- Interpretación de las correlaciones más relevantes para el análisis.

**Gráficos utilizados:**
- Matriz de correlación 

## **8. Comparación visual mediante subplots**

Se utilizan subplots para facilitar la comparación de diferentes análisis en una misma figura.

- Representación simultánea de varias distribuciones.
- Comparación de patrones entre distintas variables o grupos.
- Mejora de la interpretación visual de los resultados.

**Gráficos utilizados:**
- Subplots  
- Histogramas  
- Boxplots  
- Scatter 

## **9. Detección de valores atípicos**

Se identifican posibles valores extremos que pueden influir en el análisis.

- Localización de outliers en las variables más relevantes.
- Evaluación del impacto de estos valores en las conclusiones.
- Reflexión sobre si deben mantenerse o tratarse de forma específica.

**Gráficos utilizados:**
- Boxplots  

## **10. Conclusiones e insights finales**

En este último apartado se resumen los principales resultados obtenidos.

- Síntesis de los patrones y tendencias detectadas.
- Interpretación de los resultados obtenidos a partir de los gráficos.
- Extracción de insights útiles para la toma de decisiones.
- Propuestas de mejora o recomendaciones estratégicas basadas en el análisis.